In [10]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import seaborn
from matplotlib import pyplot
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
hotel = pd.read_csv("hotels_in_ethiopia_data.csv")
restaurant = pd.read_csv("sample-data-Ethiopian_restaurants.csv")
attraction = pd.read_csv("attractions_in_ethiopia_edited_data.csv")

In [12]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Hotel                126 non-null    object
 1   Location             126 non-null    object
 2   Star                 126 non-null    int64 
 3   Price Per Night      126 non-null    object
 4   Restaurant           126 non-null    object
 5   Free Breakfast       125 non-null    object
 6   Pizza                126 non-null    object
 7   Burger               126 non-null    object
 8   Table Tennis         126 non-null    object
 9   Horseback Riding     126 non-null    object
 10  Bicycle Riding       126 non-null    object
 11  Board Games          126 non-null    object
 12  Swimming Pool        126 non-null    object
 13  Free Cancellation    126 non-null    object
 14  Free Wifi            126 non-null    object
 15  Gym                  126 non-null    object
 16  Playstat

In [13]:
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           10 non-null     object 
 1   address        10 non-null     object 
 2   phone          10 non-null     object 
 3   email          2 non-null      object 
 4   lat            10 non-null     float64
 5   lng            10 non-null     float64
 6   url            7 non-null      object 
 7   star_count     10 non-null     float64
 8   rating_count   10 non-null     int64  
 9   category_name  10 non-null     object 
dtypes: float64(3), int64(1), object(6)
memory usage: 928.0+ bytes


In [14]:
attraction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Attraction              480 non-null    object 
 1   Location                480 non-null    object 
 2   Location_subcategory    480 non-null    object 
 3   Latitude                232 non-null    float64
 4   Longitude               232 non-null    float64
 5   Description             338 non-null    object 
 6   Activities              148 non-null    object 
 7   Offer_price             148 non-null    object 
 8   Offer_primary_category  148 non-null    object 
 9   Ranking_position        480 non-null    int64  
 10  Ranking_string          480 non-null    object 
 11  Rating                  480 non-null    float64
 12  Raw_ranking             480 non-null    float64
 13  Attraction_subcategory  480 non-null    object 
 14  Attraction_subtype      480 non-null    ob

# DATA CLEANING AND FEATURE ENGINEERING

In [15]:
del restaurant['email']
del restaurant['phone']
del attraction['Image_url']
del attraction['Offer_price']
del attraction['Offer_primary_category']


In [16]:
hotel = hotel.dropna()
restaurant = restaurant.dropna()

In [17]:
# Merge the hotel and restaurant DataFrames using the 'location' column
merged_data = pd.merge(hotel, attraction, on='Location', how='outer')
merged_data = merged_data.drop(['Star', 'Price Per Night', 'Amenities', 'Location_subcategory', 'Latitude',
       'Longitude', 'Description', 'Ranking_position',
       'Ranking_string', 'Rating', 'Raw_ranking', 'Attraction_subcategory',
       'Attraction_subtype', 'Website'], axis=1)

In [18]:
merged_data = merged_data[['Location', 'Hotel', 'Attraction', 'Activities']]
merged_data.head()

,Location,Hotel,Attraction,Activities
0,Addis Ababa,Capital Hotel And Spa,Holy Trinity Cathedral,Explore the Danakil Depression in 6 Days
1,Addis Ababa,Capital Hotel And Spa,Red Terror Martyrs Memorial Museum,Impress Addis City Tours
2,Addis Ababa,Capital Hotel And Spa,Mount Entoto,Full Day Tour of Addis Ababa with Hotel Pickup...
3,Addis Ababa,Capital Hotel And Spa,Mercato Market,Full Day Tour of Addis Ababa with Hotel Pickup...
4,Addis Ababa,Capital Hotel And Spa,St. Georges Cathedral,Full Day Tour of Addis Ababa with Hotel Pickup...


In [19]:
#removing punctuations and changing to lower case
hotel['Amenities'] = hotel['Amenities'].str.replace(',', '')
hotel['Amenities'] = hotel['Amenities'].str.lower()
hotel.head()

,Hotel,Location,Star,Price Per Night,Restaurant,Free Breakfast,Pizza,Burger,Table Tennis,Horseback Riding,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,Telephone,Refrigerator,Showerchair,Amenities
0,Capital Hotel And Spa,Addis Ababa,5,"$11,400.00",Restaurant,Free Breakfast,,,,,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,,Refrigerator,,restaurant free breakfast swimming...
1,Sheraton Addis Hotel,Addis Ababa,5,"$12,990.00",Restaurant,Free Breakfast,Pizza,Burger,,,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,Telephone,Refrigerator,Showerchair,restaurant free breakfast pizza burger bic...
2,Hilton Hotel,Addis Ababa,5,"$12,996.00",Restaurant,Free Breakfast,,,Table Tennis,,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,,,Showerchair,restaurant free breakfast table tennis b...
3,Radisson Hotel,Addis Ababa,5,"$12,450.00",Restaurant,,,,Table Tennis,,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,,,Showerchair,restaurant table tennis swimming p...
4,Ethiopian Skylight,Addis Ababa,5,"$7,600.00",Restaurant,Free Breakfast,Pizza,Burger,,Horseback Riding,...,Parkinglot,Television,Doublebed,Twinbed,Airconditioner,Bathroom,Telephone,Refrigerator,,restaurant free breakfast pizza burger horse...


# CITY-BASED HOTEL RECOMMENDER SYSTEM

In [20]:
def citybased(Location):
    hotel['Location']=hotel['Location'].str.lower()
    citybase=hotel[hotel['Location']==Location.lower()]
    citybase=citybase.sort_values(by='Star',ascending=False)
    if(citybase.empty==0):
        hname=citybase[['Hotel', 'Location', 'Star', 'Price Per Night', 'Amenities']]
        return hname.head()
    else:
        print('No Hotels Available')
        
print('Top hotels around me')
citybased('mekele')

Top hotels around me


,Hotel,Location,Star,Price Per Night,Amenities
30,Romanat Hotel,mekele,5,"$2,580.00",restaurant free breakfast horseback ridi...
24,Atse Yohannes Hotel,mekele,4,"$2,260.00",restaurant free breakfast swimming...
25,Mekelle Hotel,mekele,4,"$1,713.00",restaurant free wifi ...
26,Desta International Hotel,mekele,4,"$2,399.00",restaurant free breakfast swimming...
27,Planet Hotel,mekele,4,"$3,874.00",restaurant free breakfast swimming...


# CITY-BASED RESTAURANT RECOMMENDER SYSTEM

In [21]:
#def citybased(Location):
    #restaurant['Location']=restaurant['Location'].str.lower()
    #citybase=restaurant[restaurant['Location']==Location.lower()]
    #citybase=citybase.sort_values(by='Star',ascending=False)
    #if(citybase.empty==0):
        #rname=citybase[['Restaurant', 'Location', 'Star']]
        #return rname.head()
    #else:
        #print('No Restaurants Available')
        
#print('Top restaurants around me')
#citybased('addis ababa')

In [ ]:
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
user_location = input("Enter the city name for hotel recommendations: ")  # Get user input for the location

querystring = {"cityName": user_location, "countryName": "Ethiopia"}


In [52]:
import requests

url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"

querystring = {"cityName":"mekele","countryName":"Ethiopia"}

headers = {
	"X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",
	"X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'name': 'Homeland Hotel Mekelle', 'link': 'https://www.booking.com/hotel/et/homeland-mekelle.de.html?aid=1938431', 'rating': 9.5}


In [53]:
import requests

# API endpoint and query parameters
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
user_location = input("Enter the city name for hotel recommendations: ")  # Get user input for the location

querystring = {"cityName": user_location, "countryName": "Ethiopia"}

# Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

# Send a GET request to the hotel data API
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    hotel_data = response.json()
    
    if "hotels" in hotel_data:
        # Extract and display hotel recommendations
        for hotel in hotel_data["hotels"]:
            print(f"Hotel Name: {hotel.get('name', 'N/A')}")
            print(f"Location: {hotel.get('location', 'N/A')}")
            print(f"Rating: {hotel.get('rating', 'N/A')}")
            print(f"Amenities: {', '.join(hotel.get('amenities', []))}")
            print(f"Price per Night: {hotel.get('price_per_night', 'N/A')} USD")
            print(f"Image URL: {hotel.get('image_url', 'N/A')}")
            print("-" * 40)
    else:
        print("No hotel recommendations available for this location.")
else:
    print(f"Failed to fetch hotel recommendations. Error code: {response.status_code}")
    print(response.text)  # Print the response for debugging


Enter the city name for hotel recommendations: mekele
No hotel recommendations available for this location.


In [46]:
import requests

# API endpoint and query parameters
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
querystring = {"cityName": "Addis Ababa", "countryName": "Ethiopia"}

# Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

# Define query parameters based on the user's input (not used in this context)
# You can add query parameters specific to the API if needed

# Send a GET request to the hotel data API
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    # Parse the JSON response from the API
    hotel_data = response.json()

    # Extract and display hotel recommendations
    for hotel in hotel_data['Hotel']:
        print(f"Hotel Name: {hotel['name']}")
        print(f"Location: {hotel['location']}")
        print(f"Rating: {hotel['rating']}")
        print(f"Amenities: {', '.join(hotel['amenities'])}")
        print(f"Price per Night: {hotel['price_per_night']} USD")
        print(f"Image URL: {hotel['image_url']}")
        print("-" * 40)
else:
    print('Failed to fetch hotel recommendations. Please try again later.')


KeyError: 'Hotel'